# Урок 4
Эта демонстрация разбита на 3 ноутбука:

1. **Свертки и пулинги.**
2. Даталоадеры.
3. Задача классификации с использованием CNN.

## Свертки и пулинги в PyTorch

Рассмотрим работу Conv2d и Pooling в PyTorch.

### Свертки

In [ ]:
import torch
import torch.nn as nn

# В nn лежит Conv2d и Conv1d.
# Conv2d работает с картниками и его ядро является квадратом.
# Conv1d работает с последовательностями и его ядро является отрезком.
conv_layer = nn.Conv2d(
    in_channels=1,
    # число ядер
    out_channels=1,
    kernel_size=2,
    stride=1,
    padding=0,
    dilation=1,
    bias=False,
)
# Явно проставим веса в ядре
with torch.no_grad():
    conv_layer.weight = nn.Parameter(
        # ones
        # 1 1
        # 1 1

        # eye
        # 1 0
        # 0 1

        # ядро
        # 2 1
        # 1 2
        torch.ones((1, 1, 2, 2)) + torch.eye(2)[None, None, ...]
    )
data = torch.arange(3 * 3, dtype=torch.float32).reshape((1, 1, 3, 3))
print(data)
print(conv_layer.weight)
# Сравним выход с тем, что подсчитаем вручную.
print(conv_layer(data))

tensor([[[[0., 1., 2.],
          [3., 4., 5.],
          [6., 7., 8.]]]])
Parameter containing:
tensor([[[[2., 1.],
          [1., 2.]]]], requires_grad=True)
tensor([[[[12., 18.],
          [30., 36.]]]], grad_fn=<ConvolutionBackward0>)


In [36]:
# Свертка реализована еще и как отдельная операция
import torch.nn.functional as F

F.conv2d(data, weight=torch.ones((1, 1, 2, 2), requires_grad=True) + torch.eye(2)[None, None, ...])
# Слой Conv2d - это, по сути, обертка над F.conv2d

tensor([[[[12., 18.],
          [30., 36.]]]], grad_fn=<ConvolutionBackward0>)

### Pooling

Пулингов в PyTorch много:
- `MaxPool2d` - пройтись по ядру и взять максимум;
- `AvgPool2d` - пройтись по ядру и взять среднее;
- `AdaptiveMaxPool2d` - пройтись по **всей** картинке и взять максимум;
- `AdaptiveAvgPool2d` - пройтись по **всей** картинке и взять среднее.

Рассмотрим `MaxPool2d`, остальные аналогичны.

In [37]:
max_pool = nn.MaxPool2d(kernel_size=(2, 2))
print(data)
print(max_pool(data))
# В пулинге stride == kernel_size по умолчанию

tensor([[[[0., 1., 2.],
          [3., 4., 5.],
          [6., 7., 8.]]]])
tensor([[[[4.]]]])


In [38]:
bigger_data = torch.arange(4 * 4, dtype=torch.float32).reshape((1, 1, 4, 4))
print(bigger_data)
print(max_pool(bigger_data))

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])
tensor([[[[ 5.,  7.],
          [13., 15.]]]])


In [47]:
# Посмотрим, как работает nn.AdaptiveMaxPool2d

# исходные данные
bigger_data = torch.arange(4 * 4, dtype=torch.float32).reshape((1, 1, 4, 4))
print("Исходные данные:")
print(bigger_data)
print(f"Форма: {bigger_data.shape}\n")

# Пример 1: Уменьшение размера до (2, 2)
adaptive_pool_small = nn.AdaptiveMaxPool2d((2, 2))
output_small = adaptive_pool_small(bigger_data)
print("После AdaptiveMaxPool2d до (2, 2):")
print(output_small)
print(f"Форма: {output_small.shape}\n")

# Пример 2: Увеличение размера до (6, 6)
adaptive_pool_large = nn.AdaptiveMaxPool2d((6, 6))
output_large = adaptive_pool_large(bigger_data)
print("После AdaptiveMaxPool2d до (6, 6):")
print(output_large)
print(f"Форма: {output_large.shape}\n")

# Пример 3: Использование с одним измерением (сохраняя пропорции)
adaptive_pool_aspect = nn.AdaptiveMaxPool2d((5, None))  # None сохраняет пропорции
output_aspect = adaptive_pool_aspect(bigger_data)
print("После AdaptiveMaxPool2d до (5, None):")
print(output_aspect)
print(f"Форма: {output_aspect.shape}")

Исходные данные:
tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])
Форма: torch.Size([1, 1, 4, 4])

После AdaptiveMaxPool2d до (2, 2):
tensor([[[[ 5.,  7.],
          [13., 15.]]]])
Форма: torch.Size([1, 1, 2, 2])

После AdaptiveMaxPool2d до (6, 6):
tensor([[[[ 0.,  1.,  1.,  2.,  3.,  3.],
          [ 4.,  5.,  5.,  6.,  7.,  7.],
          [ 4.,  5.,  5.,  6.,  7.,  7.],
          [ 8.,  9.,  9., 10., 11., 11.],
          [12., 13., 13., 14., 15., 15.],
          [12., 13., 13., 14., 15., 15.]]]])
Форма: torch.Size([1, 1, 6, 6])

После AdaptiveMaxPool2d до (5, None):
tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.],
          [12., 13., 14., 15.]]]])
Форма: torch.Size([1, 1, 5, 4])


In [44]:
nn.AdaptiveMaxPool2d?

Init signature:
nn.AdaptiveMaxPool2d(
    output_size: Union[int, NoneType, Tuple[Optional[int], ...]],
    return_indices: bool = False,
) -> None
Docstring:     
Applies a 2D adaptive max pooling over an input signal composed of several input planes.

The output is of size :math:`H_{out} \times W_{out}`, for any input size.
The number of output features is equal to the number of input planes.

Args:
    output_size: the target output size of the image of the form :math:`H_{out} \times W_{out}`.
                 Can be a tuple :math:`(H_{out}, W_{out})` or a single :math:`H_{out}` for a
                 square image :math:`H_{out} \times H_{out}`. :math:`H_{out}` and :math:`W_{out}`
                 can be either a ``int``, or ``None`` which means the size will be the same as that
                 of the input.
    return_indices: if ``True``, will return the indices along with the outputs.
                    Useful to pass to nn.MaxUnpool2d. Default: ``False``

Shape:
    - Input: :